In [125]:
# Import Libraries and Dependencies
import pandas as pd

### Read in files

In [126]:
# Read in data and index by CustomerID
jan = pd.read_csv('Resources/Jan2019_sales.csv', index_col='order_ID')
feb = pd.read_csv('Resources/Feb2019_sales.csv', index_col='order_ID')
mar = pd.read_csv('Resources/March2019_sales.csv', index_col='order_ID')

In [127]:
# Show sample of January sales data.
jan.head(5)

,product,quantity_ordered,price,order_date,state,city,zip_code
order_ID,,,,,,,
141234,iPhone,1,700.00,01/22/19 21:25,FL,Orlando,32807
141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,FL,Miami,33137
141236,Wired Headphones,2,11.99,01/17/19 13:33,GA,Atlanta,30318
141237,27in FHD Monitor,1,149.99,01/05/19 20:33,CA,Los Angeles,90014
141238,Wired Headphones,1,11.99,01/25/19 11:59,CA,San Francisco,94016


In [128]:
# Show sample of February sales data.
feb.head(5)

,product,quantity_ordered,price,order_date,state,city,zip_code
order_ID,,,,,,,
150502,iPhone,1,700.00,02/18/19 01:35,PA,Philadelphia,19132
150503,AA Batteries (4-pack),1,3.84,02/13/19 07:24,CA,San Diego,91911
150504,27in 4K Gaming Monitor,1,389.99,02/18/19 09:46,CA,San Diego,91911
150505,Lightning Charging Cable,1,14.95,02/02/19 16:47,FL,Orlando,32822
150506,AA Batteries (4-pack),2,3.84,02/28/19 20:32,WA,Seattle,98115


In [129]:
# Show sample of March sales data.
mar.head(5)

,product,quantity_ordered,price,order_date,state,city,zip_code
order_ID,,,,,,,
162009,iPhone,1,700.00,03/28/19 20:59,CA,Los Angeles,90008
162009,Lightning Charging Cable,1,14.95,03/28/19 20:59,TX,Austin,78729
162009,Wired Headphones,2,11.99,03/28/19 20:59,OR,Portland,97209
162010,Bose SoundSport Headphones,1,99.99,03/17/19 05:39,FL,Miami,33125
162011,34in Ultrawide Monitor,1,379.99,03/10/19 00:01,CA,San Diego,92029


### Check the data types of each DataFrame

In [130]:
# Jan sales datatypes
jan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9681 entries, 141234 to 150501
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product           9681 non-null   object 
 1   quantity_ordered  9681 non-null   int64  
 2   price             9681 non-null   float64
 3   order_date        9681 non-null   object 
 4   state             9681 non-null   object 
 5   city              9681 non-null   object 
 6   zip_code          9681 non-null   int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 605.1+ KB


In [131]:
# Feb sales datatypes
feb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11986 entries, 150502 to 162008
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product           11986 non-null  object 
 1   quantity_ordered  11986 non-null  int64  
 2   price             11986 non-null  float64
 3   order_date        11986 non-null  object 
 4   state             11986 non-null  object 
 5   city              11986 non-null  object 
 6   zip_code          11986 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 749.1+ KB


In [132]:
# March sales datatypes
mar.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15154 entries, 162009 to 176557
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product           15154 non-null  object 
 1   quantity_ordered  15154 non-null  int64  
 2   price             15154 non-null  float64
 3   order_date        15154 non-null  object 
 4   state             15154 non-null  object 
 5   city              15154 non-null  object 
 6   zip_code          15154 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 947.1+ KB


### Combine the sales data by rows.

In [133]:
# Combine the January, February, and March sales DataFrames on the rows using an inner join, and reset the index
sales = pd.concat([jan, feb,mar], axis='rows',join = 'inner', ignore_index=False).reset_index()

#unit test - check if joined properly
# sum of all data frame count
total_count = jan.shape[0] + feb.shape[0] + mar.shape[0]  

# Count after all data frame combined 
# Assuming 'sales' is a DataFrame or any other data structure with rows representing sales data

# Get the total number of rows in the 'sales' DataFrame
total_joined_count = sales.shape[0]   

# Check if the total number of rows in 'sales' matches the expected total count
if total_count == total_joined_count:
    # If the counts match, print a message indicating successful joining
    print(f"Joined properly. Total count: {total_count} == Joined count: {total_joined_count}")
else:
    # If the counts do not match, print an error message indicating a problem in joining
    print(f'Error on joining. Total count: {total_count} != Joined count: {total_joined_count}')


sales.head(2)

Joined properly. Total count: 36821 == Joined count: 36821


,order_ID,product,quantity_ordered,price,order_date,state,city,zip_code
0,141234,iPhone,1,700.00,01/22/19 21:25,FL,Orlando,32807
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,FL,Miami,33137


In [134]:
# Check if any values are null.
sales.isna().sum()


order_ID            0
product             0
quantity_ordered    0
price               0
order_date          0
state               0
city                0
zip_code            0
dtype: int64

There are no missing values, no need for removing or imputing 

In [135]:
# Check the data type of each column
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36821 entries, 0 to 36820
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_ID          36821 non-null  int64  
 1   product           36821 non-null  object 
 2   quantity_ordered  36821 non-null  int64  
 3   price             36821 non-null  float64
 4   order_date        36821 non-null  object 
 5   state             36821 non-null  object 
 6   city              36821 non-null  object 
 7   zip_code          36821 non-null  int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 2.2+ MB


In [136]:
# Convert the "order_date" column to a datetime datatype.
sales['order_date'] = pd.to_datetime(sales['order_date'], format='%m/%d/%y %H:%M')




In [137]:
# Confirm that the data type has been changed.
sales['order_date'].head()

0   2019-01-22 21:25:00
1   2019-01-28 14:15:00
2   2019-01-17 13:33:00
3   2019-01-05 20:33:00
4   2019-01-25 11:59:00
Name: order_date, dtype: datetime64[ns]

### 1. Which top five zip codes in which cities and states had the greatest number of products ordered?

### Using `groupby`

In [138]:
# Show the average number items ordered for each state, city, and zip code.
def agg_values(df, cat_col, num_col, stat):
    """
    Aggregates data based on categorical columns and numerical columns using a specified aggregation function.

    Args:
        df (DataFrame): The input DataFrame.
        cat_col (str or list): Categorical column(s) to group by.
        num_col (str): Numerical column to aggregate.
        stat (str or dict): Type of aggregation. Can be a string representing a common aggregation function 
                            (e.g., 'mean', 'sum') or a dictionary specifying column-specific aggregation functions.

    Returns:
        DataFrame: Aggregated DataFrame with categorical columns and the specified aggregated numerical column.

    Example:
        df = pd.DataFrame({'Category': ['A', 'B', 'A', 'B'],
                           'Value': [10, 20, 30, 40]})
        result = agg_values(df, 'Category', 'Value', 'sum')
    """
    # Group by the specified categorical column(s) and aggregate the numerical column using the specified aggregation function
    aggs = df.groupby(cat_col).agg({num_col: stat})

    # Sort the aggregated DataFrame by the numerical column in descending order
    aggs_sorted = aggs.sort_values(num_col, ascending=False)

    # Reset the index to make the resulting DataFrame more readable
    return aggs_sorted


avg_items_ordered = agg_values(sales, ['state','city','zip_code'],'quantity_ordered','mean')
# Rename the sum column to reflect the data in the column.
avg_items_ordered.rename(columns={'quantity_ordered':'avg_items_ordered'},inplace= True)


# Show the top 5 results.
round(avg_items_ordered,2).head()



,,,avg_items_ordered
state,city,zip_code,
IL,Chicago,60629,1.17
CA,San Francisco,94102,1.15
FL,Miami,33125,1.15
MI,Detroit,48215,1.15
CA,San Francisco,94016,1.15


### Using `pivot_table`

In [139]:
# Show the total number of items ordered for each state, city, and zip code.
def agg_pivot(df, cat_col, num_col, stat):
    """
    Creates a pivot table by aggregating data based on categorical columns and numerical columns.

    Args:
        df (DataFrame): The input DataFrame.
        cat_col (str or list): Categorical column(s) to be used as index in the pivot table.
        num_col (str): Numerical column to aggregate.
        stat (str or dict): Type of aggregation. Can be a string representing a common aggregation function 
                            (e.g., 'mean', 'sum') or a dictionary specifying column-specific aggregation functions.

    Returns:
        DataFrame: Pivot table with categorical columns as index and the specified aggregated numerical column.

    Example:
        df = pd.DataFrame({'Category': ['A', 'B', 'A', 'B'],
                           'Value': [10, 20, 30, 40]})
        result = agg_pivot(df, 'Category', 'Value', 'sum')
    """
    # Create a pivot table using the specified categorical column(s) as index, numerical column for values, and aggregation function
    pivot_table = pd.pivot_table(df, values=num_col, index=cat_col, aggfunc=stat)

    # Sort the pivot table by the aggregated numerical column in descending order
    sorted_pivot_table = pivot_table.sort_values(num_col, ascending=False)

    # Reset the index to make the resulting DataFrame more readable
    return sorted_pivot_table

#---------------------------------------------------------------------------------------------------

items_ordered_pivot = agg_pivot(sales,['state','city','zip_code'],'quantity_ordered','sum')


# Rename the "sum" column to "Avg_Number_Products_Ordered"

items_ordered_pivot.rename(columns={'quantity_ordered':'total_items_ordered'},inplace= True)

# Show the top 5 results.
items_ordered_pivot.head()


total_items_ordered
state city          zip_code                     
IL    Chicago       60629                     909
CA    San Diego     92029                     873
                    91911                     870
NY    New York City 10001                     866
OH    Columbus      43201                     866

### 2. Which top five zip codes in which cities and states generated the most sales?

### Using `groupby`

In [140]:
# Show the total price for the items ordered for each state, city, and zip code.
sales['total_price'] = sales['price']* sales['quantity_ordered']

total_sales_price = agg_values(sales,['state','city','zip_code'],'total_price','sum')

#Format price
total_sales_price['total_price'] = total_sales_price['total_price'].map('${:,.2f}'.format)

# Rename the sum column to reflect the data in the column.
total_sales_price.rename(columns={'total_price':'total_sales_price'},inplace= True)

# Show the top 5 results.

total_sales_price.head()


total_sales_price
state city          zip_code                  
NY    Buffalo       14201          $156,582.83
IL    Chicago       60629          $151,721.31
CA    San Diego     92029          $151,342.97
      San Francisco 94102          $146,945.58
NY    New York City 10013          $146,367.28

### Using `pivot_table`

In [141]:
# Show the total price for all of the items ordered for each state, city, and zip code.
total_sales_pivot = agg_pivot(sales,['state','city','zip_code'],'total_price','sum')

# Rename the "sum" column to "Avg_Number_Products_Ordered"

total_sales_pivot['total_price'] = total_sales_pivot['total_price'].map('${:,.2f}'.format)


# Show the top 5 results.
total_sales_pivot.head()


total_price
state city          zip_code             
NY    Buffalo       14201     $156,582.83
IL    Chicago       60629     $151,721.31
CA    San Diego     92029     $151,342.97
      San Francisco 94102     $146,945.58
NY    New York City 10013     $146,367.28

### 3. Which top five zip codes in which cities and states had the greatest average price for the products ordered?

### Using `groupby`

In [142]:
# Show the average price of items ordered for each state, city, and zip code rounded to two decimal places.
avg_price = agg_values(sales,['state','city','zip_code'],'price','mean')

# Rename the mean column to reflect the data in the column.
avg_price.rename(columns={'price':'avg_price_per_product'},inplace= True)

#Round and format price
avg_price['avg_price_per_product'] = round(avg_price['avg_price_per_product'],2)
avg_price['avg_price_per_product'] = avg_price['avg_price_per_product'].map('${:,.2f}'.format)

# Show the top 5 results.
avg_price.head()

,,,avg_price_per_product
state,city,zip_code,
NY,New York City,10013,$212.11
CA,San Francisco,94102,$210.85
TX,Austin,78745,$205.63
NY,Buffalo,14201,$203.89
CA,San Diego,92029,$196.73


### Using `pivot_table`

In [143]:
# Show the average price per product ordered for each state, city, and zip code rounded to two decimal places.

avg_price_pivot = agg_pivot(sales,['state','city','zip_code'],'price','mean')

# Rename the "sum" column to "Avg_Number_Products_Ordered"
avg_price_pivot['price'] = avg_price_pivot['price'].map('${:,.2f}'.format)
avg_price_pivot.rename(columns={'price':'avg_price_per_product'},inplace= True)

# Show the top 5 results.
avg_price_pivot.head()



,,,avg_price_per_product
state,city,zip_code,
NY,New York City,10013,$212.11
CA,San Francisco,94102,$210.85
TX,Austin,78745,$205.63
NY,Buffalo,14201,$203.89
CA,San Diego,92029,$196.73


### 4. Which top five zip codes in which cities and states had the most iPhone sales?

In [144]:
# Filter the sales data to get the iPhone sales data.
iphone_sales = sales[sales['product'].str.contains('iphone',case=False)]

#print head
iphone_sales.head(5)

,order_ID,product,quantity_ordered,price,order_date,state,city,zip_code,total_price
0,141234,iPhone,1,700.0,2019-01-22 21:25:00,FL,Orlando,32807,700.0
104,141336,iPhone,1,700.0,2019-01-09 18:23:00,IL,Chicago,60629,700.0
164,141394,iPhone,1,700.0,2019-01-06 16:54:00,TX,Dallas,75201,700.0
207,141437,iPhone,1,700.0,2019-01-10 15:40:00,FL,Miami,33125,700.0
228,141457,iPhone,1,700.0,2019-01-09 22:11:00,TX,Austin,78745,700.0


### Using `groupby`

In [145]:
# Show the total number of items ordered for each state, city, and zip code.
total_iphone_ordered = agg_values(iphone_sales,['state','city','zip_code'],'quantity_ordered','sum')

# Rename the count column to reflect the data in the column.
total_iphone_ordered.rename(columns={'quantity_ordered':'total_iphone_ordered'},inplace=True)

#Show the top 5 results.
total_iphone_ordered.head()

total_iphone_ordered
state city          zip_code                      
NY    New York City 10001                       40
                    10013                       36
      Buffalo       14201                       35
TX    Austin        78701                       34
CA    San Francisco 94102                       33

### Using `pivot_table`

In [146]:
# Show the total number of items ordered for each city and zip code.
total_iphone_ordered_pivot = agg_pivot(iphone_sales,['state','city','zip_code'],'quantity_ordered','sum')

# Rename the "quantity_ordered" column to "Number_Ordered"
total_iphone_ordered_pivot.rename(columns={'quantity_ordered':'total_iphone_ordered'},inplace=True)

# Show the top 5 results.
total_iphone_ordered_pivot.head()

total_iphone_ordered
state city          zip_code                      
NY    New York City 10001                       40
                    10013                       36
      Buffalo       14201                       35
TX    Austin        78701                       34
CA    San Francisco 94102                       33

### 5. Which day had the most iPhone sales?

In [147]:
# Create a pivot table with the 'order_date' as the index and the "price" as the values.
sales_iphone_pivot = iphone_sales[['order_date','total_price']]

date_price_pivot  = agg_pivot(sales_iphone_pivot, 'order_date','total_price','sum')

# Rename the "price" column to "Total_Sales"
date_price_pivot.rename(columns={'total_price':'Total_sales'},inplace=True)

# Show the table.
date_price_pivot['Total_sales'] = date_price_pivot['Total_sales'].map('${:,.2f}'.format)
date_price_pivot.head()

,Total_sales
order_date,
2019-02-18 21:01:00,"$1,400.00"
2019-01-28 14:56:00,"$1,400.00"
2019-02-17 20:17:00,"$1,400.00"
2019-02-11 11:57:00,"$1,400.00"
2019-01-13 09:25:00,"$1,400.00"


In [148]:
# Resample the pivot table into daily bins 
sales_iphone_pivot['date_dt'] = sales_iphone_pivot['order_date'].dt.date

# aggrigate to daily sales
daily_sales = agg_values(sales_iphone_pivot,'date_dt','total_price','sum')

#top 5 
daily_sales.head()

C:\Users\hafto\AppData\Local\Temp\ipykernel_20356\3014959527.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_iphone_pivot['date_dt'] = sales_iphone_pivot['order_date'].dt.date


,total_price
date_dt,
2019-03-27,18900.0
2019-02-08,16800.0
2019-03-26,16800.0
2019-01-26,16100.0
2019-03-02,16100.0


### 6.  Which week had the most iPhone sales?

In [151]:
# Resample the pivot table into weekly bins 
# and get the total sales for each week rounded to two decimal places.
sales_iphone_pivot['week'] = sales_iphone_pivot['order_date'].dt.isocalendar().week


weekly_sales = agg_values(sales_iphone_pivot,'week','total_price','sum').map('${:,.2f}'.format)
# Sort the resampled pivot table in ascending order on "Total_Sales"
weekly_sales.sort_values('total_price').head()


C:\Users\hafto\AppData\Local\Temp\ipykernel_20356\1405970658.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_iphone_pivot['week'] = sales_iphone_pivot['order_date'].dt.isocalendar().week


,total_price
week,
3,"$46,200.00"
1,"$47,600.00"
2,"$59,500.00"
8,"$67,900.00"
7,"$74,200.00"
